In [ ]:
import pickle
from bs4 import BeautifulSoup
import pandas as pd

import re

from math import ceil

from time import sleep

In [ ]:
#FOR GOOGLE COLAB ONLY
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: use options instead of chrome_options


In [ ]:
###FOR VISUAL CODE ONLY###
# from selenium import webdriver
# PATH = 'C:\Program Files (x86)\chromedriver.exe' 
# driver = webdriver.Chrome(PATH)

**We used Selenium to connect to the site and route between the relevant pages, when we reached the required page, we took the information using BeautifulSoup, and then exported the information to a csv file.**

In [ ]:
###Login###
driver.get("https://theworker.co.il")

get_in = driver.find_element_by_link_text('כניסה')
get_in.click()

sleep(0.5)

email = driver.find_element_by_id('ranker_email')
email.clear()
email.send_keys('adir4455@gmail.com')

sleep(0.5)

password = driver.find_element_by_id('ranker_password')
password.clear()
password.send_keys('123456')

sleep(0.5)

login = driver.find_element_by_xpath('//*[@id="top"]/table/tbody/tr[2]/td/table/tbody/tr[2]/td[1]/form/table/tbody/tr[8]/td/input')
login.click()

sleep(0.5)

opnions = driver.find_element_by_link_text('חוות דעת')
opnions.click()

sleep(0.5)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() inst

In [ ]:
#Get pages
pages_html = driver.find_element_by_xpath('//*[@id="top"]/table/tbody/tr[2]/td/table/tbody/tr[3]/td[1]/div')
pages_text = pages_html.text
pages_text_to_array = pages_text.split()
items_string = pages_text_to_array[4]
items_int = int(items_string)
pages = ceil(items_int/10)

In [ ]:
###Lists for DataFrame###
ranking = []
year_of_employment = []
year_of_resignation = []
company_name = []
review_essence = []
advantages = []
disadvantages = []

for page in range(pages):
    #Wait until selector loads.
    try:
        elem = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#top > table > tbody > tr:nth-child(2) > td > table > tbody > tr:nth-child(4) > td.companies > table:nth-child(38) > tbody > tr > td > table > tbody > tr > td.nav_back > a > div")) #This is a dummy element
    )
    except:
        driver.quit()

    
    next_page = driver.find_element_by_css_selector('#top > table > tbody > tr:nth-child(2) > td > table > tbody > tr:nth-child(4) > td.companies > table:nth-child(38) > tbody > tr > td > table > tbody > tr > td.nav_back > a > div')

    page_source = driver.page_source #Load html from driver.

    #Focusing on the relevant code areas on the page, inserting the data from the site into lists
    soup = BeautifulSoup(page_source, 'lxml')
    ranking_data = soup('td', attrs={"class": re.compile("^v2BlueRatingScaleBar.")})
    years_of_employment = soup(string= re.compile('^עבד בשנים.'))
    name_of_company =soup('a',attrs={"class":"lnk_undrln"})[3::2]
    review_essence_data = soup('span',attrs={"class":'noselect'})[0::3]
    advantages_html = soup('td', attrs={"class": 'noselect'})[1::4]
    disadvantages_html = soup('td', attrs={"class": 'noselect'})[3::4]
    
    #Add ranks to list
    for rank in ranking_data:
        ranking.append(int(rank['class'][0][-2:])/10)

    #Add years to list
    for year in years_of_employment:
        years = year.split()[-1].split('-')
        emp_year = years[0]
        res_year = years[1]
        year_of_employment.append(emp_year)
        year_of_resignation.append(res_year)

    #Add name companies to list
    for company in name_of_company:
        company_name.append(company.get_text())

    #Add essence of review to list
    for review in review_essence_data:
        review_essence.append(review.get_text())

    #Add advantages to list
    for advantage in advantages_html:
        clean_sentence = advantage.get_text().replace('\n', '').replace('\r', '')
        advantages.append(clean_sentence)

    #Add disadvantages to list
    for disadvantage in disadvantages_html:
        clean_sentence = disadvantage.get_text().replace('\n', '').replace('\r', '')
        disadvantages.append(clean_sentence)

    
    next_page.click() #Next page



<ipython-input-211-6ee34f919e92>:19: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  next_page = driver.find_element_by_css_selector('#top > table > tbody > tr:nth-child(2) > td > table > tbody > tr:nth-child(4) > td.companies > table:nth-child(38) > tbody > tr > td > table > tbody > tr > td.nav_back > a > div')


In [ ]:
#Lists into raw data
raw_data = {'company_name': company_name, 'year_of_employment':year_of_employment,'year_of_resignation': year_of_resignation, 'ranking': ranking, 'review_essence': review_essence, 'disadvantages':disadvantages,'advantages':advantages}

#Raw data into df
df = pd.DataFrame(raw_data)

###Export to csv###
# df.to_csv('whole_economy_data.csv', encoding='utf-8-sig')

,company_name,year_of_employment,year_of_resignation,ranking,review_essence,disadvantages,advantages
0,אקספריס,2019,2021,4.6,מקום טוב ללמוד בו גיוס טכנולוגי,-אין צוות סורסינג,- נגיש לרכבת- איזון בית עבודה- בונוסים שווים-א...
1,אפליקאסטר,2011,2019,3.1,"אנשים טובים , טכנולוגיה סבירה , יציבות בעייתית",החברה עולה ויורדת מבחינת יציבותהנחתות של המשקי...,"אנשים טובים , תנאים לא רעים למה שזה , אופציה ט..."
2,yad 2,2018,2019,1.0,עבורי-חוויה נוראית.,•שכר נמוך אפילו ביחס לתפקיד התחלתי•סביבת עבודה...,"כעובדת במוקד לא נתקלתי ב""הטבות""."
3,איתוראן,2021,2021,2.5,אחלה של חברה עם אחלה תנאים הבעיה היא שלא משקיע...,אחלה חברה הבעיה היא בהנהלה לדעתי (ספציפית בשיר...,"עבודה מאתגרת ומגוונת, לקוחות מגוונים"
4,סייברזן,2021,2021,3.0,חברה נהדרת. חוויה מצויינת,אין! האמת יש רק דברים טובים.,"אנשים טובים, יחס חם"
...,...,...,...,...,...,...,...
995,888,2020,2021,4.4,מקום טוב ללמוד בו,שכר סביר ביחס לשוק,"אחלה אנשים, אווירה טובה ללמידה והתפתחות, גם לת..."
996,עיריית תל אביב,2021,2021,3.8,העירייה הכי טובה בארץ,אין חסרונות בעיריית תל אביב,אנשים טובים ונחמדים
997,בי איי סיינס,2017,2020,4.2,חברה עם אווירה של משפחה,לא רוצה לרשום חסרונות,ביקוש גבוה למוצרי החברה
998,"צה""ל",2012,2021,3.3,ארגון גדול שלא בהכרח מחזיק בטובים ביותר,שעות עבודה לא נוחותנסיעות ארוכותשכר לא גבוה,ביטחון תעסוקתישכר גולבליהחזר גני ילדים
